In [1]:
!pip install torch


  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.7.3 requires numpy<2.0.0, but you have numpy 2.2.6 which is incompatible.
vllm 0.7.3 requires transformers>=4.48.2, but you have transformers 4.42.1 which is incompatible.


In [2]:
# Task 1: Load and preprocess League of Legends data

# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import os

# Load the dataset
file_path = os.path.expanduser('./league_of_legends_data_large.csv')
df = pd.read_csv(file_path)

# Confirm the column names to ensure the target column exists
print("Columns:", df.columns)

# Replace 'blueWins' with the correct column name if different
target_column = 'blueWins'

# Ensure the target column exists
if target_column not in df.columns:
    raise ValueError(f"Target column '{target_column}' not found in dataset. Please update the code with the correct column name.")

# Split into features and target
X = df.drop(columns=target_column)
y = df[target_column]

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Check final shapes
print("X_train_tensor shape:", X_train_tensor.shape)
print("y_train_tensor shape:", y_train_tensor.shape)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import pandas as pd
import os

file_path = os.path.expanduser('~/Downloads/league_of_legends_data_large.csv')
df = pd.read_csv(file_path)

# Show all column names
print(df.columns.tolist())


In [ ]:
# Task 1: Load and preprocess the League of Legends dataset

import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

# Load dataset
file_path = os.path.expanduser('~/Downloads/league_of_legends_data_large.csv')
df = pd.read_csv(file_path)

# Define target column correctly
target_column = 'win'

# Split into features and target
X = df.drop(columns=target_column)
y = df[target_column]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

print("✅ Data loaded and preprocessed successfully.")


In [ ]:
# Task 2: Define logistic regression model using PyTorch

import torch.nn as nn
import torch.optim as optim

# Define the logistic regression model class
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

# Set input dimension (number of features)
input_dim = X_train_tensor.shape[1]

# Initialize the model
model = LogisticRegressionModel(input_dim)

# Define the loss function (Binary Cross-Entropy Loss)
loss_fn = nn.BCELoss()

# Define the optimizer (SGD with learning rate 0.01)
optimizer = optim.SGD(model.parameters(), lr=0.01)

print("✅ Logistic Regression model, loss function, and optimizer initialized.")


In [ ]:
# Task 3: Model Training

# Number of epochs
num_epochs = 1000

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = loss_fn(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Print loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Model evaluation
model.eval()
with torch.no_grad():
    train_preds = model(X_train_tensor).squeeze()
    test_preds = model(X_test_tensor).squeeze()

    train_preds_class = (train_preds >= 0.5).float()
    test_preds_class = (test_preds >= 0.5).float()

    train_accuracy = (train_preds_class == y_train_tensor).sum().item() / y_train_tensor.size(0)
    test_accuracy = (test_preds_class == y_test_tensor).sum().item() / y_test_tensor.size(0)

print(f"\n✅ Training Accuracy: {train_accuracy:.4f}")
print(f"✅ Test Accuracy: {test_accuracy:.4f}")


In [ ]:
# Reinitialize the model
model_l2 = LogisticRegressionModel(input_dim)
# Set up the optimizer with L2 regularization
optimizer_l2 = optim.SGD(model_l2.parameters(), lr=0.01, weight_decay=0.01)
loss_fn_l2 = nn.BCELoss()

# Train the model with L2 regularization
num_epochs = 1000
for epoch in range(num_epochs):
    model_l2.train()
    optimizer_l2.zero_grad()

    outputs = model_l2(X_train_tensor)
    loss = loss_fn_l2(outputs, y_train_tensor)

    loss.backward()
    optimizer_l2.step()

    if (epoch + 1) % 100 == 0:
        print(f"[L2] Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the L2-regularized model
model_l2.eval()
with torch.no_grad():
    train_preds_l2 = model_l2(X_train_tensor).squeeze()
    test_preds_l2 = model_l2(X_test_tensor).squeeze()

    train_preds_class_l2 = (train_preds_l2 >= 0.5).float()
    test_preds_class_l2 = (test_preds_l2 >= 0.5).float()

    train_accuracy_l2 = (train_preds_class_l2 == y_train_tensor).sum().item() / y_train_tensor.size(0)
    test_accuracy_l2 = (test_preds_class_l2 == y_test_tensor).sum().item() / y_test_tensor.size(0)

print(f"\n✅ L2-Regularized Training Accuracy: {train_accuracy_l2:.4f}")
print(f"✅ L2-Regularized Test Accuracy: {test_accuracy_l2:.4f}")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, auc

# Ensure model is in evaluation mode
model_l2.eval()

# Get predicted probabilities
with torch.no_grad():
    y_test_probs = model_l2(X_test_tensor).squeeze()
    y_test_pred = (y_test_probs >= 0.5).float()

# 1. Confusion Matrix
cm = confusion_matrix(y_test_tensor.numpy(), y_test_pred.numpy())
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# 2. Classification Report
print("Classification Report:\n")
print(classification_report(y_test_tensor.numpy(), y_test_pred.numpy(), digits=4))

# 3. ROC Curve and AUC
fpr, tpr, thresholds = roc_curve(y_test_tensor.numpy(), y_test_probs.numpy())
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.grid()
plt.show()


In [ ]:
# Save the trained model (state dict only)
torch.save(model_l2.state_dict(), "logistic_model_l2.pth")

# Re-instantiate a new model with the same architecture
loaded_model = LogisticRegressionModel(input_dim)
loaded_model.load_state_dict(torch.load("logistic_model_l2.pth"))
loaded_model.eval()

# Evaluate the loaded model on test data
with torch.no_grad():
    loaded_outputs = loaded_model(X_test_tensor).squeeze()
    loaded_predictions = (loaded_outputs >= 0.5).float()

# Calculate and print accuracy
loaded_accuracy = (loaded_predictions == y_test_tensor).float().mean()
print(f"Accuracy of loaded model on test set: {loaded_accuracy:.4f}")


In [ ]:
learning_rates = [0.01, 0.05, 0.1]
num_epochs = 100
best_accuracy = 0
best_lr = None

for lr in learning_rates:
    print(f"\nTraining with learning rate: {lr}")

    # Reinitialize model and optimizer for each learning rate
    model = LogisticRegressionModel(input_dim)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor).squeeze()
        loss = criterion(outputs, y_train_tensor.squeeze())
        loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor).squeeze()
        predictions = (outputs >= 0.5).float()
        accuracy = (predictions == y_test_tensor).float().mean().item()

    print(f"Test accuracy for learning rate {lr}: {accuracy:.4f}")

    # Track best
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_lr = lr

print(f"\n✅ Best learning rate: {best_lr} with accuracy: {best_accuracy:.4f}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Extract weights from the trained logistic regression model
weights = model.linear.weight.data.numpy().flatten()

# Step 2: Create DataFrame with feature names and their weights
feature_names = X_train.columns  # Assuming you have access to column names here
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': weights
})

# Step 3: Sort features by absolute importance
importance_df['AbsImportance'] = importance_df['Importance'].abs()
importance_df = importance_df.sort_values(by='AbsImportance', ascending=False)

# Step 4: Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Weight (Importance)')
plt.title('Feature Importance from Logistic Regression Model')
plt.gca().invert_yaxis()  # Most important on top
plt.tight_layout()
plt.show()
